# Exploring the Melbourne Bike Share Dataset with DuckDB

## Getting the dataset

https://melbournetestbed.opendatasoft.com/explore/dataset/melbourne-bike-share-station-readings-2011-2017/information/

In [ ]:
%%bash
pip install -r requirements.txt
wget --no-clobber https://opendatasoft-s3.s3.amazonaws.com/downloads/archive/74id-aqj9.zip
unzip -n 74id-aqj9.zip
mkdir -p data
mv 74id-aqj9.csv data/melb_bike_share.csv

## Loading the dataset with Pandas

In [ ]:
%%time
import pandas as pd

bikes_df = pd.read_csv(
    "data/melb_bike_share.csv",
    usecols=["ID", "NAME", "NBBIKES", "RUNDATE", "LAT", "LONG"],
    parse_dates=["RUNDATE"],
    date_format="%Y%m%d%H%M%S",
)

bikes_df

## Loading the dataset with DuckDB

Key observations
* compare and contrast with Pandas equivalent
* result is a relation object (separate slide on relations)

In [ ]:
%%time
import duckdb

bikes_rel = duckdb.sql(
    """
    SELECT ID, NAME, NBBIKES, RUNDATE, LAT, LONG
    FROM read_csv( 
        'data/melb_bike_share.csv', 
        types={'RUNDATE': TIMESTAMP},
        timestampformat='%Y%m%d%H%M%S'
    )
    """
)

bikes_rel.show(max_rows=10)

# Querying the dataset

Goal: get the average number of bikes across all stations by month for 2017.

Notable callouts:
* replacement scanning of 'rel' variable
* lazy evaluation: result not materialised until we print the output

In [ ]:
%%time
monthly_bikes_rel = duckdb.sql(
    """
    SELECT 
        month(RUNDATE) AS MONTH,
        round(avg(NBBIKES), 2) AS AVG_BIKES,
    FROM bikes_rel
    WHERE year(RUNDATE) = 2017
    GROUP BY MONTH
    ORDER BY MONTH
    """
)

monthly_bikes_rel

## Pandas version of the query

Notable callouts:
* more inscrutable than the SQL
* results have to be materialised at each method call

In [ ]:
avg_bikes_df = (
    bikes_df[bikes_df["RUNDATE"].dt.year == 2017]
    .groupby(bikes_df["RUNDATE"].dt.month)["NBBIKES"]
    .mean()
    .round(2)
    .reset_index(name="AVG_BIKES")
    .sort_values("RUNDATE")
)

avg_bikes_df

## Exporting our dataset back to disk

In [ ]:
duckdb.sql("COPY monthly_bikes_rel TO 'data/monthly_avg_bikes.csv'")

In [ ]:
duckdb.sql("COPY monthly_bikes_rel TO 'data/monthly_avg_bikes.parquet'")

In [ ]:
duckdb.sql("COPY monthly_bikes_rel TO 'data/monthly_avg_bikes.json'")

Passing in options:

In [ ]:
duckdb.sql("COPY monthly_bikes_rel TO 'data/monthly_avg_bikes.csv' (DELIMITER '|')")

In [ ]:
duckdb.sql("COPY monthly_bikes_rel TO 'data/monthly_avg_bikes.json' (ARRAY true)")

### Some nice SQL features

In [ ]:
duckdb.sql(
    """    
    COPY (
        SELECT * FROM bikes_rel LIMIT 100000
    ) TO 'bike_share_100k.csv'"
    """
)

In [ ]:
duckdb.sql(
    """    
    COPY (
        FROM bikes_rel USING SAMPLE 100000
    ) TO 'bike_share_sample_100k.csv'
    """
)

In [ ]:
duckdb.sql("COPY (FROM bikes_rel USING SAMPLE 100000) TO 'bike_share_sample.csv'")

In [ ]:
duckdb.sql("FROM rel LIMIT 3")

## Python in-memory data format integration

### Exporting

In [ ]:
bikes_rel.df()

In [ ]:
bikes_rel.pl()

In [ ]:
bikes_rel.arrow()

In [ ]:
bikes_rel.fetchnumpy()

### Importing

In [ ]:
duckdb.sql("SELECT * FROM pandas_df")

In [ ]:
duckdb.sql("SELECT * FROM polars_df")

In [ ]:
duckdb.sql("SELECT * FROM arrow_table")

In [ ]:
duckdb.sql("SELECT * FROM numpy_array")

## Data viz

In [ ]:
monthly_bikes_df = monthly_bikes_rel.df()

In [ ]:
monthly_bikes_df

In [ ]:
monthly_bikes_df.plot.line(x="MONTH", y="AVG_BIKES")

In [ ]:
px.line

In [ ]:
import plotly.express as px

px.line(monthly_bikes_df, x="MONTH", y="AVG_BIKES")

In [ ]:
%%time
duckdb.sql("FROM rel USING SAMPLE 3")

### Converting to Pandas and Polars Dataframes

In [ ]:
%%time
df = rel.df()

### Querying a dataframe

In [ ]:
%%time
duckdb.sql("FROM df USING SAMPLE 3")

--------------------------

In [ ]:
%%time
conn.sql(
    """
    CREATE OR REPLACE TABLE bikes AS
    FROM relation
    """
)

### JupySQL for convenient querying in Jupyter Notebooks

In [ ]:
# load JupySQL extension requried for the %sql and %%sql magics
%load_ext sql 

# register the DuckDB connection with JupySQL 
%sql conn --alias duckdb 

# configure JupySQL to return Pandas DataFrames by default
%config SqlMagic.autopandas = True 

In [ ]:
%%sql 
SELECT 
    NAME,
    extract('year' FROM RUNDATE) AS YEAR,
    extract('month' FROM RUNDATE) AS MONTH,
    COUNT(*) AS NUM_READINGS,
FROM bikes
WHERE MONTH = 1
GROUP BY YEAR, MONTH, NAME
ORDER BY YEAR, MONTH

In [ ]:
station_reading_counts_df

In [ ]:
import plotly.express as px 

px.line(
    station_reading_counts_df,
    x="MONTH",
    y="NUM_READINGS",
    markers=True, 
    symbol="YEAR",
    symbol_sequence=["square", "diamond", "circle"],
    color="YEAR",
    title="Records by month for each year the Melbourne Bike Share program was active",
    height=400,
).update_traces(marker_size=8)

## Working with the Relational API

In [ ]:
conn.table("bikes").describe()

In [ ]:
bikes_rel = conn.table("bikes")

In [ ]:
bikes_rel.project("RUNDATE", "NBBIKES", "NBEMPTYDOCKS").describe()

In [ ]:
bikes_rel.filter("LOCKED = true").value_counts("NAME").order("2 DESC")

# Scratch

In [ ]:
%%sql
SELECT count(DISTINCT LOCATION) FROM bikes

In [ ]:
%%sql
SELECT count(DISTINCT NAME) FROM bikes

In [ ]:
%%sql
SELECT count(LOCATION)
FROM bikes
GROUP BY LOCATION

# Ad-hoc wrangling

In [ ]:
rel

In [ ]:
duckdb.sql("COPY (FROM rel USING SAMPLE 1000000) TO 'sample_bikes.csv'")

# In-Memory databases vs Persistant file database

In [ ]:
mem_conn = duckdb.connect()

In [ ]:
file_conn = duckdb.connect("bike_share.duckdb")